# Template_Matching
1.img 는 도로의 이미지입니다.<br/>
(park_1.png, park_2.png, park_3.png, road_1jpg, road_2.jpg)<br/>
2. template은 표지판 이미지입니다 각 template는 도로이미지에서 따왔습니다.<br/> 
(ex_park1.png, ex_park2.png, ex_park3.png, parking_sign.png, slipery_road1.png, ex_slip.png)

'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF_NORMED' 이 3개는 서로 다른 비교방법 입니다.<br/>
(https://cord-ai.tistory.com/86) 이거 보시면 알 것 같아요

CCOEFF = 같으면 1, 연관성 없으면 0, 역일치 -1<br/>
CCORR = 같으면 큰값, 다르면 작은 값<br/>
SQDIFF = 같으면 0, 다르면 값이큼<br/>
하지만 보기편하게 백분률로 바꿔놨습니다.


매칭 결과는 사각형 으로 표시가 됩니다.

img 내에서 template를 찾는건데 이미지 크기도 중요한 것 같습니다.<br/>
(https://076923.github.io/posts/Python-opencv-37/) 이거 보시면 될 것 같습니다.

In [40]:
# 템플릿 매칭으로 객체 위치 검출 (template_matching.py)

import cv2
import numpy as np

# 입력이미지와 템플릿 이미지 읽기
img = cv2.imread('./img/park2.png')
#img = cv2.resize(img, (640, 480))
template = cv2.imread('./img/ex_park1.png')
th, tw = template.shape[:2]
cv2.imshow('template', template)

# 3가지 매칭 메서드 순회
methods = ['cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR_NORMED', \
                                     'cv2.TM_SQDIFF_NORMED']
for i, method_name in enumerate(methods):
    img_draw = img.copy()
    method = eval(method_name)
    # 템플릿 매칭   ---①
    res = cv2.matchTemplate(img, template, method)
    # 최솟값, 최댓값과 그 좌표 구하기 ---②
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    print(method_name, min_val, max_val, min_loc, max_loc)

    # TM_SQDIFF의 경우 최솟값이 좋은 매칭, 나머지는 그 반대 ---③
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
        match_val = round((1 - min_val), 3) * 100
    else:
        top_left = max_loc
        match_val = round(max_val, 3) * 100
    # 매칭 좌표 구해서 사각형 표시   ---④      
    bottom_right = (top_left[0] + tw, top_left[1] + th)
    cv2.rectangle(img_draw, top_left, bottom_right, (0,0,255),2)
    # 매칭 포인트 표시 ---⑤
    cv2.putText(img_draw, str(match_val) + "%", top_left, \
                cv2.FONT_HERSHEY_PLAIN, 2,(0,255,0), 1, cv2.LINE_AA)
    cv2.imshow(method_name, img_draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.TM_CCOEFF_NORMED -0.3776586055755615 0.27199849486351013 (445, 168) (4, 212)
cv2.TM_CCORR_NORMED 0.6908811330795288 0.9495624303817749 (1, 505) (668, 60)
cv2.TM_SQDIFF_NORMED 0.10346263647079468 1.0 (668, 71) (201, 324)


# MultiScale_Template_Matching

위 코드와 다른점은 template이미지를 scales로 크기를 바꿔가면서 img와 비교하는 알고리즘입니다.<br/>
Template Matching 알고리즘의 단점인 template이미지 사이즈 이슈를 해결할 수 있는 것 같아요.

cv2.COLOR_BGR2GRAY는 cv2.COLOR_BGR2GRAY, cv2.COLOR_BGR2HSV, cv2.COLOR_BGR2RGB 이렇게 바꿔가면서 비교해봐도 좋습니다.

In [41]:
import cv2
import numpy as np

def multi_scale_template_matching(img, template):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    
    methods = ['cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF_NORMED']
    
    for i, method_name in enumerate(methods):
        img_draw = img.copy()
        method = eval(method_name)
        
        # 다양한 크기의 템플릿 생성
        scales = [0.3, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]
        
        for scale in scales:
            resized_template = cv2.resize(template_gray, (int(template.shape[1]*scale), int(template.shape[0]*scale)))
            
            res = cv2.matchTemplate(img_gray, resized_template, method)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

            if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                top_left = min_loc
                match_val = round((1 - min_val), 3) * 100
            else:
                top_left = max_loc
                match_val = round(max_val, 3) * 100
            
            bottom_right = (top_left[0] + int(template.shape[1]*scale), top_left[1] + int(template.shape[0]*scale))
            
            cv2.rectangle(img_draw, top_left, bottom_right, (0, 0, 255), 2)
            cv2.putText(img_draw, f"{scale}: {match_val} %", (top_left[0], top_left[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1, cv2.LINE_AA)
        
        cv2.imshow(f"{method_name} - Multi-Scale", img_draw)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 입력이미지와 템플릿 이미지 읽기
img = cv2.imread('./img/road_1.jpg')
img = cv2.resize(img, (640, 853))
template = cv2.imread('./img/ex_park3.png')

# Multi-Scale Template Matching 적용
multi_scale_template_matching(img, template)


#multi_scale_multi_template_matching

template이미지를 여러개 비교

(cv2.COLOR_BGR2HSV, cv2.TM_CCOEFF_NORMED)가 정확도 높았음 

In [43]:
import cv2
import numpy as np

def multi_scale_multi_template_matching(img, templates):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    methods = ['cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF_NORMED']
    
    for i, method_name in enumerate(methods):
        img_draw = img.copy()
        method = eval(method_name)
        
        max_match_val = 0  # 가장 높은 매칭 점수를 저장하는 변수
        best_scale = 0  # 가장 높은 매칭 점수를 갖는 스케일을 저장하는 변수
        best_top_left = (0, 0)  # 가장 높은 매칭 점수를 갖는 좌표를 저장하는 변수
        
        for template in templates:
            template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2HSV)
            
            # 다양한 크기의 템플릿 생성
            scales = [0.3, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]
            
            for scale in scales:
                resized_template = cv2.resize(template_gray, (int(template.shape[1]*scale), int(template.shape[0]*scale)))
                
                res = cv2.matchTemplate(img_gray, resized_template, method)
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

                if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                    match_val = round((1 - min_val), 3) * 100
                else:
                    match_val = round(max_val, 3) * 100

                if match_val > max_match_val:
                    max_match_val = match_val
                    best_scale = scale
                    best_top_left = max_loc if method not in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED] else min_loc
        
        # 매칭 좌표 구해서 사각형 표시
        best_bottom_right = (best_top_left[0] + int(template.shape[1] * best_scale), best_top_left[1] + int(template.shape[0] * best_scale))
        cv2.rectangle(img_draw, best_top_left, best_bottom_right, (0, 0, 255), 2)
        cv2.putText(img_draw, f"Best Match ({best_scale}): {max_match_val} %", (best_top_left[0], best_top_left[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1, cv2.LINE_AA)
        
        cv2.imshow(f"{method_name} - Multi-Scale Multi-Template", img_draw)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 입력이미지와 여러 템플릿 이미지 읽기
img = cv2.imread('./img/road_1.jpg')
img = cv2.resize(img, (640, 853))
template1 = cv2.imread('./img/ex_park1.png')
template2 = cv2.imread('./img/ex_park2.png')
template3 = cv2.imread('./img/ex_park3.png')

# 여러 템플릿 적용
templates = [template1, template2, template3]
multi_scale_multi_template_matching(img, templates)
